<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/PySpark/PySpark_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark - Ejercicios clase 8

### Instalación y carga de Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=c477ee1af74f033150d0c2a8e22f74db5fe0a200d42ee68b0ac808ee6c609a1a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

### Librerías necesarias

In [4]:
from pyspark.sql.types import StringType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np

### Importamos datos

In [5]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/results.csv

--2023-05-09 23:57:57--  https://data-engineer-edvai.s3.amazonaws.com/f1/results.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 3.5.10.1, 54.231.228.41, 54.231.50.75, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|3.5.10.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633624 (1.6M) [text/csv]
Saving to: ‘results.csv’

results.csv         100%[===================>]   1.56M  9.60MB/s    in 0.2s    

2023-05-09 23:57:58 (9.60 MB/s) - ‘results.csv’ saved [1633624/1633624]



In [6]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/drivers.csv

--2023-05-09 23:57:58--  https://data-engineer-edvai.s3.amazonaws.com/f1/drivers.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 3.5.10.1, 54.231.228.41, 54.231.50.75, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|3.5.10.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93568 (91K) [text/csv]
Saving to: ‘drivers.csv’

drivers.csv         100%[===================>]  91.38K  --.-KB/s    in 0.06s   

2023-05-09 23:57:58 (1.55 MB/s) - ‘drivers.csv’ saved [93568/93568]



In [7]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/constructors.csv

--2023-05-09 23:57:58--  https://data-engineer-edvai.s3.amazonaws.com/f1/constructors.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 3.5.10.1, 54.231.228.41, 54.231.50.75, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|3.5.10.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17384 (17K) [text/csv]
Saving to: ‘constructors.csv’

constructors.csv    100%[===================>]  16.98K  --.-KB/s    in 0.001s  

2023-05-09 23:57:58 (28.1 MB/s) - ‘constructors.csv’ saved [17384/17384]



In [ ]:
# Con esta alternativa algunos tipos de datos no se importan de forma adecuada
# df = spark.read.option("header","true").csv("*.csv")

In [17]:
# Con esta alternativa los tipos de datos se importan de forma adecuada (siempre verificar!)
spk_results = spark.read.option("header","true").option("inferSchema", "true") .csv("results.csv")

In [18]:
spk_drivers = spark.read.option("header","true").option("inferSchema", "true") .csv("drivers.csv")

In [19]:
spk_constructors = spark.read.option("header","true").option("inferSchema", "true") .csv("constructors.csv")

In [20]:
import pandas as pd
df_results = pd.read_csv("/content/results.csv")

In [15]:
df_results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25655,25661,1086,825,210,20,13,16,16,16,0.0,69,\N,\N,37,15,1:23.511,188.856,11
25656,25662,1086,848,3,23,17,17,17,17,0.0,69,\N,\N,43,12,1:23.047,189.911,11
25657,25663,1086,849,3,6,19,18,18,18,0.0,69,\N,\N,60,8,1:22.478,191.221,11
25658,25664,1086,852,213,22,16,19,19,19,0.0,68,\N,\N,58,16,1:23.538,188.795,12


### Visualizamos schema

In [21]:
spk_results.printSchema()

root
 |-- resultId: integer (nullable = true)
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- constructorId: integer (nullable = true)
 |-- number: string (nullable = true)
 |-- grid: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- positionText: string (nullable = true)
 |-- positionOrder: integer (nullable = true)
 |-- points: double (nullable = true)
 |-- laps: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- milliseconds: string (nullable = true)
 |-- fastestLap: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- fastestLapTime: string (nullable = true)
 |-- fastestLapSpeed: string (nullable = true)
 |-- statusId: integer (nullable = true)



In [22]:
spk_results.show(5)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|       1|    18|       1|            1|    22|   1|       1|           1|            1|  10.0|  58|1:34:50.616|     5690616|        39|   2|      1:27.452|        218.300|       1|
|       2|    18|       2|            2|     3|   5|       2|           2|            2|   8.0|  58|     +5.478|     5696094|        41|   3|      1:27.739|        217.586|       1|
|       3|    18|       3|            3|     7|   7|       3|           3|            3|  

In [23]:
spk_drivers.printSchema()

root
 |-- driverId: integer (nullable = true)
 |-- driverRef: string (nullable = true)
 |-- number: string (nullable = true)
 |-- code: string (nullable = true)
 |-- forename: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)



In [24]:
spk_drivers.show(5)

+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
|driverId| driverRef|number|code|forename|   surname|       dob|nationality|                 url|
+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|   Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|    \N| HEI|    Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|    Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO|Fernando|    Alonso|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|    \N| KOV|  Heikki|Kovalainen|1981-10-19|    Finnish|http://en.wikiped...|
+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
only showing top 5 rows



In [25]:
spk_constructors.printSchema()

root
 |-- constructorId: integer (nullable = true)
 |-- constructorRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)



In [26]:
spk_constructors.show(5)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|       mclaren|   McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber|BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|  Williams|    British|http://en.wikiped...|
|            4|       renault|   Renault|     French|http://en.wikiped...|
|            5|    toro_rosso|Toro Rosso|    Italian|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 5 rows



### Insertar en la tabla driver_results los corredores con mayor cantidad de puntos en la historia.

In [27]:
# Creamos vista
spk_results.createOrReplaceTempView("results")
spk_drivers.createOrReplaceTempView("drivers")

In [28]:
query = """SELECT results.driverId, drivers.driverRef, SUM(results.points) as total
           FROM results 
           JOIN drivers ON results.driverId = drivers.driverId
           GROUP BY results.driverId, drivers.driverRef
           ORDER BY total DESC """

df_pilotos = spark.sql(query)

In [29]:
df_pilotos.show()

+--------+------------------+------+
|driverId|         driverRef| total|
+--------+------------------+------+
|       1|          hamilton|4308.5|
|      20|            vettel|3077.0|
|       4|            alonso|2021.0|
|       8|         raikkonen|1873.0|
|     830|    max_verstappen|1792.5|
|     822|            bottas|1775.0|
|       3|           rosberg|1594.5|
|      30|michael_schumacher|1566.0|
|     817|         ricciardo|1289.0|
|      18|            button|1235.0|
|      13|             massa|1167.0|
|     815|             perez|1059.0|
|      17|            webber|1047.5|
|     117|             prost| 798.5|
|     844|           leclerc| 724.0|
|     832|             sainz| 680.5|
|      22|       barrichello| 658.0|
|     102|             senna| 614.0|
|      14|         coulthard| 535.0|
|     807|        hulkenberg| 521.0|
+--------+------------------+------+
only showing top 20 rows



### Insertar en la tabla constructor_result quienes obtuvieron más puntos en el Spanish Grand Prix en el año 1991

In [30]:
# Creamos vista
spk_constructors.createOrReplaceTempView("team")

In [33]:
query = """SELECT results.constructorId, team.constructorRef, SUM(results.points) as total
           FROM results 
           JOIN team ON results.constructorId = team.constructorId
           WHERE results.raceId=318
           GROUP BY results.constructorId, team.constructorRef
           ORDER BY total DESC """

df_SGP = spark.sql(query)

In [34]:
df_SGP.show(30)

+-------------+--------------+-----+
|constructorId|constructorRef|total|
+-------------+--------------+-----+
|            3|      williams| 14.0|
|            6|       ferrari|  9.0|
|            1|       mclaren|  2.0|
|           22|      benetton|  1.0|
|           36|     fondmetal|  0.0|
|           29|      footwork|  0.0|
|           32|    team_lotus|  0.0|
|           34|       brabham|  0.0|
|           35|       dallara|  0.0|
|           17|        jordan|  0.0|
|           40|         lambo|  0.0|
|           39|           ags|  0.0|
|           25|       tyrrell|  0.0|
|           26|          lola|  0.0|
|           27|        ligier|  0.0|
|           41|        leyton|  0.0|
|           18|       minardi|  0.0|
+-------------+--------------+-----+

